<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_naranja@4x.png?raw=true" alt="esquema" />
</div>

# Laboratorio Agrupamiento de Datos

En este laboratorio usaremos el DataFrame creado en la lección anterior resultado de la unión de los conjuntos de datos compartidos contigo. 

**Instrucciones:**

1. Lee cuidadosamente el enunciado de cada ejercicio.

2. Implementa la solución en la celda de código proporcionada.

3. Devuelve los resultados siempre en formato de DataFrame

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) 

In [129]:
df = pd.read_csv("./df_resultante.csv", index_col=0)
df.head(1)

,show_id,type,Title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,_merge
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",Documentary,2020-10-02,90.0,7.5,English,both


#### Ejercicio 1: ¿Cuántos títulos hay por cada tipo de contenido ('Movie' o 'TV Show')?

In [100]:
df1 = df.groupby("type").count()
df1

,show_id,Title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,_merge
type,,,,,,,,,,,,,,,,,
Movie,6131,6131,5943,5656,5691,6131,6131,6129,3503,6131,6131,513,513,513,513,513,6131
TV Show,2676,2676,230,2326,2285,2666,2676,2674,491,2676,2676,0,0,0,0,0,2676


#### Ejercicio 2: ¿Cuál es el promedio de puntuación de las películas agrupadas por clasificación de audiencia ('rating')? Interpreta los resultados, ¿hay algo que te llame la atención? Explica por qué. 

In [102]:
df2 = df.groupby('rating')['IMDB Score'].mean().round(2).dropna()
df2
#Poniendo el dropna() al final, primero agrupamos por rating, se calcula la media
#usando el IMDB score y si el resultado cuenta con un NaN, se excluye
#así mostramos unicamente la información que queremos

#aunque también será interesante ver porqué esos ratings no cuentan con IMDB score


rating
G        6.50
PG       6.80
PG-13    6.39
R        6.50
TV-14    6.18
TV-G     6.16
TV-MA    6.10
TV-PG    6.31
TV-Y     6.78
TV-Y7    6.24
Name: IMDB Score, dtype: float64

In [72]:
# Para ver más en profundidad este problema veamos los valores de IMDB score por rating
print("Registros por tipo de rating", df['rating'].value_counts())

print("Valores nulos en IMDB score por tipo de rating",df[df['IMDB Score'].isnull()].groupby('rating').size())

Registros por tipo de rating rating
TV-MA       3207
TV-14       2160
TV-PG        863
R            799
PG-13        490
TV-Y7        334
TV-Y         307
PG           287
TV-G         220
NR            80
G             41
TV-Y7-FV       6
NC-17          3
UR             3
74 min         1
84 min         1
66 min         1
Name: count, dtype: int64
Valores nulos en IMDB score por tipo de rating rating
66 min         1
74 min         1
84 min         1
G             40
NC-17          3
NR            80
PG           276
PG-13        467
R            752
TV-14       2066
TV-G         204
TV-MA       2953
TV-PG        806
TV-Y         302
TV-Y7        329
TV-Y7-FV       6
UR             3
dtype: int64


#### Ejercicio 3: ¿Cuántos títulos hay por cada país? Ordena los resultados de mayor a menor

In [103]:
#El problema que encontramos es que el campo pais, es un string 
# que puede estar compuesto por multiples paises separados por coma

# Como se puede ver con este ejemplo
df.iloc[7]['country']

'United States, Ghana, Burkina Faso, United Kingdom, Germany, Ethiopia'

In [104]:
# Dividimos los países en una lista
df_paises = df[['country', 'Title']].copy()
df_paises['country'] = df_paises['country'].str.split(',')

# Usando el metodo explode, cada elemento de la lista, ahora pasa a ser un registro del dataframe
df_paises = df_paises.explode('country')

# Usando value_counts, de manera descendente nos da las coincidencias por la columna country, siendo cada una 1 título
resultado = df_paises['country'].value_counts()
resultado


country
United States     3211
India             1008
United Kingdom     628
 United States     479
Canada             271
                  ... 
 Ecuador             1
Iran                 1
Cyprus               1
 Mongolia            1
 Montenegro          1
Name: count, Length: 197, dtype: int64

#### Ejercicio 4: ¿Cuál es la calificación promedio en IMDb de los títulos agrupados por tipo ('Movie' o 'TV Show')?

In [105]:
df4 = df.groupby('type', dropna=True)['IMDB Score'].mean().round(2)
df4
# Como pasó anteriormente, al trabajar con IMDB score, esta columna cuenta con muchos nulos

type
Movie      6.21
TV Show     NaN
Name: IMDB Score, dtype: float64

In [106]:
# Para ver más en profundidad este problema veamos los valores de IMDB score por tipo de contenido
print("Registros por tipo de contenido", df['type'].value_counts())

print("Valores nulos de IMDB score por tipo de contenido",df[df['IMDB Score'].isnull()].groupby('type').size())
# Aquí podemos ver el problema, como unicamente contamos con 513 valores de IMDB core en específico, en Movies

Registros por tipo de contenido type
Movie      6131
TV Show    2676
Name: count, dtype: int64
Valores nulos de IMDB score por tipo de contenido type
Movie      5618
TV Show    2676
dtype: int64


#### Ejercicio 5: ¿Cuántos títulos se añadieron a Netflix cada año? Para resolver este ejercicio tendrás uqe convertir la columna `date_added` a formato fecha usando el método `pd.to_datetime`. [Aquí](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) tienes la documentación, leeala con detenimiento antes de usar el método. 

In [107]:
print(df["date_added"].dtype) # Es de tipo objeto, string
df["date_added"].head(5)

object


0    2021-09-25
1    2021-09-24
2    2021-09-24
3    2021-09-24
4    2021-09-24
Name: date_added, dtype: object

In [109]:
# Por experimentar, crearé una copia de la columna
df['date_added_original'] = df['date_added']

In [115]:
# Aprovechando el formato, daremos el formato
df['date_added'] = pd.to_datetime(df['date_added'], format='%Y-%m-%d')

print(df['date_added'].head())

0   2021-09-25
1   2021-09-24
2   2021-09-24
3   2021-09-24
4   2021-09-24
Name: date_added, dtype: datetime64[ns]


In [114]:
print(df["date_added"].dtype) # Ahora ya es de tipo fecha
df["date_added"].head()

datetime64[ns]


0   2021-09-25
1   2021-09-24
2   2021-09-24
3   2021-09-24
4   2021-09-24
Name: date_added, dtype: datetime64[ns]

In [119]:
df5 = df.groupby(df['date_added'].dt.year)['Title'].count().sort_index()
df5

date_added
2008.0       2
2009.0       2
2010.0       1
2011.0      13
2012.0       3
2013.0      11
2014.0      24
2015.0      82
2016.0     429
2017.0    1188
2018.0    1649
2019.0    2016
2020.0    1879
2021.0    1498
Name: Title, dtype: int64

#### Ejercicio 6: ¿Cuál es el país con más títulos lanzados en el año 2020? Para resolver este ejercicio después del gropuby os puede resultar interesante el método [`idxmax`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.idxmax.html)

In [123]:
df_paises_2020 = df[['country', 'Title']].copy()
df_paises['country'] = df_paises['country'].str.split(',')
df_paises = df_paises.explode('country')
resultado = df_paises['country'].value_counts()
resultado.idxmax()


'United States'

#### Ejercicio 7:¿Cuál es el promedio de puntuación en IMDb de los títulos agrupados por país?

#### Ejercicio 8: ¿Cuántos títulos fueron lanzados en cada año agrupados por tipo ('Movie' o 'TV Show')?

In [126]:
# Group by release year and type, then count titles
# We'll sort by year to see the temporal progression
df8 = df.groupby(['release_year', 'type'])['Title'].count()
print("Distribución de títulos por año y tipo:")
print(df8)

Distribución de títulos por año y tipo:
release_year  type   
1925          TV Show      1
1942          Movie        2
1943          Movie        3
1944          Movie        3
1945          Movie        3
                        ... 
2019          TV Show    397
2020          Movie      517
              TV Show    436
2021          Movie      277
              TV Show    315
Name: Title, Length: 119, dtype: int64


#### Ejercicio 9: ¿Cuál es el país con la mayor cantidad de títulos de clasificación 'TV-MA'?

#### Ejercicio 10. ¿Cuál es el promedio de duración de las películas agrupadas por país?

In [ ]:
df4 = df.groupby('country')['IMDB Score'].mean().round(2)
df4
# Como pasó anteriormente, al trabajar con IMDB score, esta columna cuenta con muchos nulos

#### Ejercicio 11: ¿Cuál es el número total de títulos por país y clasificación de audiencia ('rating')?

#### Ejercicio 12: ¿Cuántos títulos fueron lanzados antes del año 2000 agrupados por país?

#### Ejercicio 13: ¿Cuál es la puntuación promedio en IMDb de los títulos lanzados en 2021 agrupados por país?

#### Ejercicio 14: ¿Cuántos títulos fueron dirigidos por cada director? Ordena los resultados de mayor a menor

In [132]:
df14 = df["director"].value_counts()
print("Top 5 directores con más producciones", df14.head())

Top 5 directores con más producciones director
Rajiv Chilaka             19
Raúl Campos, Jan Suter    18
Marcus Raboy              16
Suhas Kadav               16
Jay Karas                 14
Name: count, dtype: int64


#### Ejercicio 15: ¿Cuál es el lenguaje más frecuente en los títulos lanzados en los últimos 5 años agrupado por país?